In [1]:
import io
import os
import sys
import cv2
import random

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

class ImageAnnotator():

    def __init__(self, path_cred=''):

        # Set credentials
        self.path_cred = path_cred
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path_cred

        # Instantiates a client
        self.client = vision.ImageAnnotatorClient()
        self.list_used_labels = []

        # Set fix colors for labels 
#         self.list_used_labels.append(['Top', (0, 0, 255)]) #Red
        self.list_used_labels.append(['Table top', (0, 0, 255)])
#         self.list_used_labels.append(['Clothing', (0, 0, 255)])
        self.list_used_labels.append(['Building', (0, 0, 255)])

        self.list_used_labels.append(['Top', (255, 255, 255)])
        self.list_used_labels.append(['Clothing', (255, 255, 255)])

        self.list_used_labels.append(['Person', (0, 255, 0)]) #Green

        self.list_used_labels.append(['Dress', (255, 0, 0)]) #Blue

        # set parameters for rectangle
        self.thickness_rect = 1

        # set parameters for text
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.font_scale = 1
        self.line_type = 1



    def get_objects(self, path_img, print_data):

        """ 
        Annonates image at location path_img returns objects in detected in the image
        args:
            path_img = string
            print_data = bool - prints object data if true
        """ 

        # The name of the image file to annotate
#         file_name = os.path.join(
#             os.path.dirname(__file__),
#             path_img)
        file_name =  path_img

        # Loads the image into memory
        with io.open(file_name, 'rb') as image_file:
            content = image_file.read()

            image = types.Image(content=content)
            objects = self.client.object_localization(
                image=image).localized_object_annotations

        if print_data:
            print('Number of objects found: {}'.format(len(objects)))
            for object_ in objects:
                print('\n{} (confidence: {})'.format(object_.name, object_.score))
                print('Normalized bounding polygon vertices: ')
                for vertex in object_.bounding_poly.normalized_vertices:
                    print(' - ({}, {})'.format(vertex.x, vertex.y))
        return objects

    def annotate_image(self, path_img, path_img_an, img_objects):
        # read image
        img = cv2.imread(path_img)
        for object_ in img_objects:
            # get vertices
            vertices = []
            for vertex in object_.bounding_poly.normalized_vertices:
                vertices.append((int(vertex.x * img.shape[1]), int(vertex.y* img.shape[0])))

            # Set color for annotation
            color = None
            for label in self.list_used_labels:
                if label[0] == object_.name:
                    color = label[1]
            if color is None:
                # Append label with color to list
                self.list_used_labels.append([object_.name,
                    (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))])

            # Draw rectangle
            img_an = cv2.rectangle(img, vertices[0] , vertices[2], color, self.thickness_rect)

            # generate string
            str_text = '{0} - {1:.2f}'.format(object_.name, object_.score)

            # put text on the image
            cv2.putText(img_an ,str_text, vertices[0], self.font, self.font_scale, color, self.line_type)


        # save image
        cv2.imwrite(path_img_an, img_an)


In [2]:
import cv2
import os
import re
import numpy

class VideoTransformer():

    def __init__(self, fps=30):
        self.fps = fps

    def video_to_frame(self, path_video, dir_frames):

        vidcap = cv2.VideoCapture(path_video)
        success,image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite(dir_frames + "/frame_" + str(count).zfill(10) + ".jpg", image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1


    def frame_to_video(self, dir_frames, path_video, fps):
        num_img = []
        images = []
        for img in os.listdir(dir_frames):
            if img.endswith(".png") or img.endswith(".jpg"):
                # using List comprehension + isdigit() +split() 
                # getting numbers from string  
                data = img.split("_")
                data = data[1].split(".")
                num = data[0]
                #print(num)
                num_img.append(num)
                images.append(img)

        sorted_numbers = numpy.argsort(num_img)
        images_sorted = []
        #print(images)
        for i in range(len(images)):
            images_sorted.append(images[sorted_numbers[i]])

        frame = cv2.imread(os.path.join(dir_frames, images_sorted[0]))
        height, width, layers = frame.shape
        fourrc = cv2.VideoWriter_fourcc(*'MP4V')
        video = cv2.VideoWriter(path_video, fourrc, fps, (width, height))

        for image in images_sorted:
            frame = cv2.imread(os.path.join(dir_frames, image))
            video.write(frame)

        cv2.destroyAllWindows()
        video.release()

In [4]:
import os


# Set parameters
PATH_CRED = "C://Users//USER//Desktop//GOOGLE_CL_KEY//VideoDetection-503abf24e493.json"
PATH_VID = "C://Users//USER//Desktop//OBJECT_DETECTION_NEW _PROJECT//sample.mp4"# location video 
DIR_FRAMES = "D://OD//frames//"# directory to store frames (will be created)
PATH_VID_AN = "C://Users//USER//Desktop//OBJECT_DETECTION_NEW _PROJECT//sample_output_vision.mp4"# name of annotated video
DIR_FRAMES_AN = "D://OD//labeled_frames//"# directory to store annotated frames (will be created)
VID_FPS = 30

print("running")

# Create objects
img_annotator = ImageAnnotator(PATH_CRED)
video_transf = VideoTransformer(VID_FPS)

# create folder for frames
try:
    os.mkdir(DIR_FRAMES)
except:
    print("Folder " + DIR_FRAMES + " already exists")
    for filename in os.listdir(DIR_FRAMES):
        file_path = os.path.join(DIR_FRAMES, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

try:
    os.mkdir(DIR_FRAMES_AN)
except:
    print("Folder " + DIR_FRAMES_AN + " already exists")
    for filename in os.listdir(DIR_FRAMES_AN):
        file_path = os.path.join(DIR_FRAMES_AN, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Turn video into frames
print("Turn video into frames")
video_transf.video_to_frame(PATH_VID, DIR_FRAMES)

# iterate through frames
print("Get informations from images")
for file in os.listdir(DIR_FRAMES):
    if file.endswith(".jpg") or file.endswith(".png"):
        # Get information from images
        img_objects = img_annotator.get_objects(DIR_FRAMES+file, True)

        # Annotate images
        img_annotator.annotate_image(DIR_FRAMES+file, DIR_FRAMES_AN+file, img_objects)

# transform frames to video
print("Transform frames to video")
video_transf.frame_to_video(DIR_FRAMES_AN, PATH_VID_AN,VID_FPS)

print("Done!")